## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -c "import monai; import nibabel; import tqdm; import einops" || pip install -q "monai-weekly[nibabel, tqdm, einops]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import tempfile
#!pip install nibabel
import nibabel as nib
import numpy as np
import json
import pandas as pd
from scipy.stats.mstats import winsorize


import matplotlib.pyplot as plt
from tqdm import tqdm

from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.transforms import (
    AsDiscrete,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    EnsureTyped,
)

from monai.config import print_config
from monai.metrics import DiceMetric, compute_hausdorff_distance
from monai.networks.nets import SwinUNETR

from monai.data import (
    ThreadDataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
    set_track_meta,
)


import torch
import einops
from einops import rearrange
from monai.metrics import HausdorffDistanceMetric
import numpy as np
print_config()

MONAI version: 1.4.dev2350
Numpy version: 1.23.5
Pytorch version: 2.1.0+cu118
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: e54c05d7a659e4d402f5194412889779e6856cbd
MONAI __file__: /usr/local/lib/python3.10/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 4.0.2
scikit-image version: 0.19.3
scipy version: 1.11.4
Pillow version: 9.4.0
Tensorboard version: 2.14.1
gdown version: 4.6.6
TorchVision version: 0.16.0+cu118
tqdm version: 4.66.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 1.5.3
einops version: 0.7.0
transformers version: 4.35.2
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io

## Transform

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#### Transforms
num_samples = 4

train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"], ensure_channel_first=True),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-175,
            a_max=250,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        # Spacingd(
        #     keys=["image", "label"],
        #     pixdim=(1.5, 1.5, 2.0),
        #     mode=("bilinear", "nearest"),
        # ),
        EnsureTyped(keys=["image", "label"], device=device, track_meta=False),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=num_samples,
            image_key="image",
            image_threshold=0,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[0],
            prob=0.10,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[1],
            prob=0.10,
        ),
        RandFlipd(
            keys=["image", "label"],
            spatial_axis=[2],
            prob=0.10,
        ),
        RandRotate90d(
            keys=["image", "label"],
            prob=0.10,
            max_k=3,
        ),
        RandShiftIntensityd(
            keys=["image"],
            offsets=0.10,
            prob=0.50,
        ),
    ]
)
val_transforms = Compose(
[
    LoadImaged(keys=["image", "label"], ensure_channel_first=True),
    ScaleIntensityRanged(keys=["image"], a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
    CropForegroundd(keys=["image", "label"], source_key="image"),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    # Spacingd(
    #     keys=["image", "label"],
    #     pixdim=(1.5, 1.5, 2.0),
    #     mode=("bilinear", "nearest"),
    # ),
    EnsureTyped(keys=["image", "label"], device=device, track_meta=True),
]
)

/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


## Load Dataset

In [ ]:
data_dir = "/content/drive/MyDrive/Swin UNetr Data/"

def gen_val_loader(fold):
    split_json = "data2/dataset_fold_" + str(fold) + ".json"
    datasets = data_dir + split_json

    val_files = load_decathlon_datalist(datasets, True, "validation")
    val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_num=6, cache_rate=1.0, num_workers=1)
    val_loader = ThreadDataLoader(val_ds, num_workers=0, batch_size=1)

    return val_loader, val_files
# val_loader = gen_val_loader(fold = 1)

## Check data shape and visualize

In [ ]:
# slice_map = {
#     "LT_147_ds256.nii.gz": 120,
#     "RT_143_ds256.nii.gz": 120,
# }
# case_num = 2
# img_name = os.path.split(val_ds[case_num]["image"].meta["filename_or_obj"])[1]
# print(img_name)
# img = val_ds[case_num]["image"]
# label = val_ds[case_num]["label"]
# img_shape = img.shape
# label_shape = label.shape
# print(f"image shape: {img_shape}, label shape: {label_shape}")
# plt.figure("image", (18, 6))
# plt.subplot(1, 2, 1)
# plt.title("image")
# plt.imshow(img[0, :, :, slice_map[img_name]].detach().cpu(), cmap="gray")
# plt.subplot(1, 2, 2)
# plt.title("label")
# plt.imshow(label[0, :, :, slice_map[img_name]].detach().cpu())
# plt.show()

## Set up model

In [ ]:
### Set up model
def gen_model(fold):
    model = SwinUNETR(
        img_size=(96, 96, 96),
        in_channels=1,
        out_channels=17,
        feature_size=48,
        use_checkpoint=True,
    ).to(device)

    weights_f = data_dir + "fold_" + str(fold) + "_model.pth"
    weight = torch.load(weights_f)

    model.load_state_dict(weight)

    return model
model = gen_model(1)

/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)


## Hausdorff Code

In [ ]:
### Code from https://github.com/andy-s-ding/tbone-seg-nnunet
!pip install git+https://github.com/google-deepmind/surface-distance.git

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#from . import lookup_tables  # pylint: disable=relative-beyond-top-level
import numpy as np
from scipy import ndimage
import os
import sys
import argparse
import numpy as np
import pandas as pd
import nibabel as nib
import glob
import fnmatch
import surface_distance
from surface_distance.metrics import *
import time


def _assert_is_numpy_array(name, array):
    """Raises an exception if `array` is not a numpy array."""
    if not isinstance(array, np.ndarray):
        raise ValueError("The argument {!r} should be a numpy array, not a "
                        "{}".format(name, type(array)))


def _check_nd_numpy_array(name, array, num_dims):
    """Raises an exception if `array` is not a `num_dims`-D numpy array."""
    if len(array.shape) != num_dims:
        raise ValueError("The argument {!r} should be a {}D array, not of "
                        "shape {}".format(name, num_dims, array.shape))


def _check_2d_numpy_array(name, array):
    _check_nd_numpy_array(name, array, num_dims=2)


def _check_3d_numpy_array(name, array):
    _check_nd_numpy_array(name, array, num_dims=3)


def _assert_is_bool_numpy_array(name, array):
    _assert_is_numpy_array(name, array)
    if array.dtype != np.bool:
        raise ValueError("The argument {!r} should be a numpy array of type bool, "
                        "not {}".format(name, array.dtype))


def _compute_bounding_box(mask):
    """Computes the bounding box of the masks.

    This function generalizes to arbitrary number of dimensions great or equal
    to 1.

    Args:
        mask: The 2D or 3D numpy mask, where '0' means background and non-zero means
        foreground.

    Returns:
        A tuple:
        - The coordinates of the first point of the bounding box (smallest on all
        axes), or `None` if the mask contains only zeros.
        - The coordinates of the second point of the bounding box (greatest on all
        axes), or `None` if the mask contains only zeros.
    """
    num_dims = len(mask.shape)
    bbox_min = np.zeros(num_dims, np.int64)
    bbox_max = np.zeros(num_dims, np.int64)

    # max projection to the x0-axis
    proj_0 = np.amax(mask, axis=tuple(range(num_dims))[1:])
    idx_nonzero_0 = np.nonzero(proj_0)[0]
    if len(idx_nonzero_0) == 0:  # pylint: disable=g-explicit-length-test
        return None, None

    bbox_min[0] = np.min(idx_nonzero_0)
    bbox_max[0] = np.max(idx_nonzero_0)

    # max projection to the i-th-axis for i in {1, ..., num_dims - 1}
    for axis in range(1, num_dims):
        max_over_axes = list(range(num_dims))  # Python 3 compatible
        max_over_axes.pop(axis)  # Remove the i-th dimension from the max
        max_over_axes = tuple(max_over_axes)  # numpy expects a tuple of ints
        proj = np.amax(mask, axis=max_over_axes)
        idx_nonzero = np.nonzero(proj)[0]
        bbox_min[axis] = np.min(idx_nonzero)
        bbox_max[axis] = np.max(idx_nonzero)

    return bbox_min, bbox_max


def _crop_to_bounding_box(mask, bbox_min, bbox_max):
    """Crops a 2D or 3D mask to the bounding box specified by `bbox_{min,max}`."""
    # we need to zeropad the cropped region with 1 voxel at the lower,
    # the right (and the back on 3D) sides. This is required to obtain the
    # "full" convolution result with the 2x2 (or 2x2x2 in 3D) kernel.
    # TODO:  This is correct only if the object is interior to the
    # bounding box.
    cropmask = np.zeros((bbox_max - bbox_min) + 2, np.uint8)

    num_dims = len(mask.shape)
    # pyformat: disable
    if num_dims == 2:
        cropmask[0:-1, 0:-1] = mask[bbox_min[0]:bbox_max[0] + 1,
                                    bbox_min[1]:bbox_max[1] + 1]
    elif num_dims == 3:
        cropmask[0:-1, 0:-1, 0:-1] = mask[bbox_min[0]:bbox_max[0] + 1,
                                        bbox_min[1]:bbox_max[1] + 1,
                                        bbox_min[2]:bbox_max[2] + 1]
    # pyformat: enable
    else:
        assert False

    return cropmask


def _sort_distances_surfels(distances, surfel_areas):
    """Sorts the two list with respect to the tuple of (distance, surfel_area).

    Args:
        distances: The distances from A to B (e.g. `distances_gt_to_pred`).
        surfel_areas: The surfel areas for A (e.g. `surfel_areas_gt`).

    Returns:
        A tuple of the sorted (distances, surfel_areas).
    """
    sorted_surfels = np.array(sorted(zip(distances, surfel_areas)))
    return sorted_surfels[:, 0], sorted_surfels[:, 1]

def compute_surface_distances(mask_gt,
                              mask_pred,
                              spacing_mm):
    """Computes closest distances from all surface points to the other surface.

    This function can be applied to 2D or 3D tensors. For 2D, both masks must be
    2D and `spacing_mm` must be a 2-element list. For 3D, both masks must be 3D
    and `spacing_mm` must be a 3-element list. The description is done for the 2D
    case, and the formulation for the 3D case is present is parenthesis,
    introduced by "resp.".

    Finds all contour elements (resp surface elements "surfels" in 3D) in the
    ground truth mask `mask_gt` and the predicted mask `mask_pred`, computes their
    length in mm (resp. area in mm^2) and the distance to the closest point on the
    other contour (resp. surface). It returns two sorted lists of distances
    together with the corresponding contour lengths (resp. surfel areas). If one
    of the masks is empty, the corresponding lists are empty and all distances in
    the other list are `inf`.

    Args:
        mask_gt: 2-dim (resp. 3-dim) bool Numpy array. The ground truth mask.
        mask_pred: 2-dim (resp. 3-dim) bool Numpy array. The predicted mask.
        spacing_mm: 2-element (resp. 3-element) list-like structure. Voxel spacing
        in x0 anx x1 (resp. x0, x1 and x2) directions.

    Returns:
        A dict with:
        "distances_gt_to_pred": 1-dim numpy array of type float. The distances in mm
            from all ground truth surface elements to the predicted surface,
            sorted from smallest to largest.
        "distances_pred_to_gt": 1-dim numpy array of type float. The distances in mm
            from all predicted surface elements to the ground truth surface,
            sorted from smallest to largest.
        "surfel_areas_gt": 1-dim numpy array of type float. The length of the
        of the ground truth contours in mm (resp. the surface elements area in
        mm^2) in the same order as distances_gt_to_pred.
        "surfel_areas_pred": 1-dim numpy array of type float. The length of the
        of the predicted contours in mm (resp. the surface elements area in
        mm^2) in the same order as distances_gt_to_pred.

    Raises:
        ValueError: If the masks and the `spacing_mm` arguments are of incompatible
        shape or type. Or if the masks are not 2D or 3D.
    """
    # The terms used in this function are for the 3D case. In particular, surface
    # in 2D stands for contours in 3D. The surface elements in 3D correspond to
    # the line elements in 2D.

    _assert_is_bool_numpy_array("mask_gt", mask_gt)
    _assert_is_bool_numpy_array("mask_pred", mask_pred)

    if not len(mask_gt.shape) == len(mask_pred.shape) == len(spacing_mm):
        raise ValueError("The arguments must be of compatible shape. Got mask_gt "
                        "with {} dimensions ({}) and mask_pred with {} dimensions "
                        "({}), while the spacing_mm was {} elements.".format(
                            len(mask_gt.shape),
                            mask_gt.shape, len(mask_pred.shape), mask_pred.shape,
                            len(spacing_mm)))

    num_dims = len(spacing_mm)
    if num_dims == 2:
        _check_2d_numpy_array("mask_gt", mask_gt)
        _check_2d_numpy_array("mask_pred", mask_pred)

        # compute the area for all 16 possible surface elements
        # (given a 2x2 neighbourhood) according to the spacing_mm
        neighbour_code_to_surface_area = (
            lookup_tables.create_table_neighbour_code_to_contour_length(spacing_mm))
        kernel = lookup_tables.ENCODE_NEIGHBOURHOOD_2D_KERNEL
        full_true_neighbours = 0b1111
    elif num_dims == 3:
        _check_3d_numpy_array("mask_gt", mask_gt)
        _check_3d_numpy_array("mask_pred", mask_pred)

        # compute the area for all 256 possible surface elements
        # (given a 2x2x2 neighbourhood) according to the spacing_mm
        neighbour_code_to_surface_area = (
            lookup_tables.create_table_neighbour_code_to_surface_area(spacing_mm))
        kernel = lookup_tables.ENCODE_NEIGHBOURHOOD_3D_KERNEL
        full_true_neighbours = 0b11111111
    else:
        raise ValueError("Only 2D and 3D masks are supported, not "
                        "{}D.".format(num_dims))

    # compute the bounding box of the masks to trim the volume to the smallest
    # possible processing subvolume
    bbox_min, bbox_max = _compute_bounding_box(mask_gt | mask_pred)
    # Both the min/max bbox are None at the same time, so we only check one.
    if bbox_min is None:
        return {
            "distances_gt_to_pred": np.array([]),
            "distances_pred_to_gt": np.array([]),
            "surfel_areas_gt": np.array([]),
            "surfel_areas_pred": np.array([]),
        }

    # crop the processing subvolume.
    cropmask_gt = _crop_to_bounding_box(mask_gt, bbox_min, bbox_max)
    cropmask_pred = _crop_to_bounding_box(mask_pred, bbox_min, bbox_max)

    # compute the neighbour code (local binary pattern) for each voxel
    # the resulting arrays are spacially shifted by minus half a voxel in each
    # axis.
    # i.e. the points are located at the corners of the original voxels
    neighbour_code_map_gt = ndimage.filters.correlate(
        cropmask_gt.astype(np.uint8), kernel, mode="constant", cval=0)
    neighbour_code_map_pred = ndimage.filters.correlate(
        cropmask_pred.astype(np.uint8), kernel, mode="constant", cval=0)

    # create masks with the surface voxels
    borders_gt = ((neighbour_code_map_gt != 0) &
                    (neighbour_code_map_gt != full_true_neighbours))
    borders_pred = ((neighbour_code_map_pred != 0) &
                    (neighbour_code_map_pred != full_true_neighbours))

    # compute the distance transform (closest distance of each voxel to the
    # surface voxels)
    if borders_gt.any():
        distmap_gt = ndimage.morphology.distance_transform_edt(
            ~borders_gt, sampling=spacing_mm)
    else:
        distmap_gt = np.Inf * np.ones(borders_gt.shape)

    if borders_pred.any():
        distmap_pred = ndimage.morphology.distance_transform_edt(
            ~borders_pred, sampling=spacing_mm)
    else:
        distmap_pred = np.Inf * np.ones(borders_pred.shape)

    # compute the area of each surface element
    surface_area_map_gt = neighbour_code_to_surface_area[neighbour_code_map_gt]
    surface_area_map_pred = neighbour_code_to_surface_area[
        neighbour_code_map_pred]

    # create a list of all surface elements with distance and area
    distances_gt_to_pred = distmap_pred[borders_gt]
    distances_pred_to_gt = distmap_gt[borders_pred]
    surfel_areas_gt = surface_area_map_gt[borders_gt]
    surfel_areas_pred = surface_area_map_pred[borders_pred]

    # sort them by distance
    if distances_gt_to_pred.shape != (0,):
        distances_gt_to_pred, surfel_areas_gt = _sort_distances_surfels(
            distances_gt_to_pred, surfel_areas_gt)

    if distances_pred_to_gt.shape != (0,):
        distances_pred_to_gt, surfel_areas_pred = _sort_distances_surfels(
            distances_pred_to_gt, surfel_areas_pred)

    return {
        "distances_gt_to_pred": distances_gt_to_pred,
        "distances_pred_to_gt": distances_pred_to_gt,
        "surfel_areas_gt": surfel_areas_gt,
        "surfel_areas_pred": surfel_areas_pred,
    }

def compute_average_surface_distance(surface_distances):
    """Returns the average surface distance.

    Computes the average surface distances by correctly taking the area of each
    surface element into account. Call compute_surface_distances(...) before, to
    obtain the `surface_distances` dict.

    Args:
        surface_distances: dict with "distances_gt_to_pred", "distances_pred_to_gt"
        "surfel_areas_gt", "surfel_areas_pred" created by
        compute_surface_distances()

    Returns:
        A tuple with two float values:
        - the average distance (in mm) from the ground truth surface to the
            predicted surface
        - the average distance from the predicted surface to the ground truth
            surface.
    """
    distances_gt_to_pred = surface_distances["distances_gt_to_pred"]
    distances_pred_to_gt = surface_distances["distances_pred_to_gt"]
    surfel_areas_gt = surface_distances["surfel_areas_gt"]
    surfel_areas_pred = surface_distances["surfel_areas_pred"]
    average_distance_gt_to_pred = (
        np.sum(distances_gt_to_pred * surfel_areas_gt) / np.sum(surfel_areas_gt))
    average_distance_pred_to_gt = (
        np.sum(distances_pred_to_gt * surfel_areas_pred) /
        np.sum(surfel_areas_pred))
    return (average_distance_gt_to_pred, average_distance_pred_to_gt)

#Space finding
def find_space(path):
    gt_nii = nib.load(path)
    gt_header = gt_nii.header
    spacing = np.asarray(gt_header.get_zooms())
    return spacing


  Cloning https://github.com/google-deepmind/surface-distance.git to /tmp/pip-req-build-teswaxgi
  Running command git clone --filter=blob:none --quiet https://github.com/google-deepmind/surface-distance.git /tmp/pip-req-build-teswaxgi
  Resolved https://github.com/google-deepmind/surface-distance.git to commit ee651c80e11ab5fcc3d1b0c1c6e7bbc6fb4c987c
  Preparing metadata (setup.py) ... done
  Created wheel for Surface-Distance-Based-Measures: filename=Surface_Distance_Based_Measures-0.1-py3-none-any.whl size=14339 sha256=68a29420dba4b5c82ff6e986a6ab86a5316609200da60798f91b180801be29ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-vz44c77m/wheels/7f/71/a6/91987e1a77d5970790f95b3006d1d17ff55a5d8e862888199e
Successfully built Surface-Distance-Based-Measures


## Validation

In [ ]:
post_label = AsDiscrete(to_onehot=17)
post_pred = AsDiscrete(argmax=True, to_onehot=17)
dice_metric = DiceMetric(include_background=True, reduction="none", get_not_nans=False)
hausdorff_distance_metric = HausdorffDistanceMetric(include_background=True, reduction="none")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def val_metrics(val_loader, json_validation):
    model.eval()
    with torch.no_grad():
        dice_scores = []
        hausdorff_distances_all = []
        for index, batch in enumerate(val_loader):
            val_inputs, val_labels = (batch["image"].cuda(), batch["label"].cuda())
            with torch.cuda.amp.autocast():
                val_outputs = sliding_window_inference(val_inputs, (96, 96, 96), 2, model)
            val_labels_list = decollate_batch(val_labels)
            val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]
            val_outputs_list = decollate_batch(val_outputs)
            val_output_convert = [post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list]

            #Compute dice loss for each label
            dice_metric = DiceMetric(include_background=True, reduction="none", get_not_nans=False)
            # print(f"pred:{val_output_convert[0].shape}")
            # print(f"label:{val_labels_convert[0].shape}")
            dice_metric(y_pred=val_output_convert, y=val_labels_convert)
            individual_scores = dice_metric.aggregate().cpu().numpy()
            dice_scores.append(individual_scores)
            dice_metric.reset()
            print("")
            print(f"{index}th validation image Dice scores:, {individual_scores}")


            # #Compute hausdorff distances for each label
            # for pred, label in zip(val_output_convert, val_labels_convert):
            #     hausdorff_distance_metric(y_pred=pred.unsqueeze(0), y=label.unsqueeze(0))
            #     hausdorff_distance = hausdorff_distance_metric.aggregate().cpu().numpy()
            #     hausdorff_distances.append(hausdorff_distance)
            #     #   hausdorff_distance = compute_hausdorff_distance(y_pred=pred.unsqueeze(0), y=label.unsqueeze(0), spacing = 0.1923)
            #     #   hausdorff_distances.append(hausdorff_distance)
            #     hausdorff_distance_metric.reset()
            # print("")
            # print(f"{index}the validation image hausdorff distances:, {hausdorff_distance}")

            pred =val_output_convert[0].cpu().numpy()
            label = val_labels_convert[0].cpu().numpy()
            pred = np.array(pred,dtype=bool)
            gt = np.array(label,dtype=bool)
            hausdorff_distances =[]

            #Find voxel spacing
            path = json_validation[index]["label"]
            print(path)
            space = find_space(path)

            #Extract hausdorff distance for each class
            for index2 in range(pred.shape[0]):
                surface_distances = compute_surface_distances(gt[index2,:,:], pred[index2,:,:], space)
                hausdorff_distance = max(compute_average_surface_distance(surface_distances))
                hausdorff_distances.append(hausdorff_distance)
            hausdorff_distances_all.append(hausdorff_distances)

            print("")
            print(f"{index}th validation image hausdorff distances:, {hausdorff_distances}")


    return np.array(dice_scores), np.array(hausdorff_distances_all)

In [ ]:
dice = np.empty((0,17))
HD = np.empty((0,17))

for i in range(5):
    fold = i + 1
    print("----------------")
    print("Generating Statistics for Fold {}".format(fold))
    print("----------------")
    val_loader, val_files = gen_val_loader(fold = fold)
    model = gen_model(fold = fold)
    dice_i, HD_i = val_metrics(val_loader, val_files)
    dice = np.vstack((dice, dice_i.reshape(-1,17)))
    HD = np.vstack((HD, HD_i.reshape(-1,17)))
    del model, val_loader


----------------
Generating Statistics for Fold 1
----------------


Loading dataset: 100%|██████████| 4/4 [00:06<00:00,  1.63s/it]



0th validation image Dice scores:, [[0.94482416 0.9360417  0.8680954  0.8739345  0.5300462  0.9010545
  0.7897116  0.6798122  0.23759398 0.78115904 0.7360814  0.09080326
  0.8809288  0.7238196  0.34497178 0.9279081  0.7724637 ]]
/content/drive/MyDrive/Swin UNetr Data/data2/labelsTr/Segmentation_RT_147_ds256.nii.gz

0th validation image hausdorff distances:, [0.20064912985933833, 0.0391367920162251, 0.058677299172434805, 0.6256600759933965, 0.423149667112035, 0.14103744290318435, 0.4208611740053269, 0.3699972967660443, 0.97821269090994, 0.12668202985518098, 0.9534338288026891, 3.1213382574029147, 0.2965756724945393, 0.8228993307525091, 0.7097492021160781, 0.24099538101656148, 0.7839638871363634]

1th validation image Dice scores:, [[0.96025556 0.9279723  0.88717633 0.8874232  0.6328125  0.9290801
  0.88267183 0.51720047 0.3856041  0.65717494 0.80349636 0.1560035
  0.94286203 0.77369404 0.2676303  0.93571997 0.8879722 ]]
/content/drive/MyDrive/Swin UNetr Data/data2/labelsTr/Segmentation

Loading dataset: 100%|██████████| 4/4 [00:06<00:00,  1.62s/it]



0th validation image Dice scores:, [[0.9634804  0.9543088  0.80324656 0.8362622  0.         0.86552715
  0.859635   0.6931507  0.39263803 0.77138066 0.8249557  0.
  0.82184404 0.7643173  0.40647483 0.9318863  0.85712606]]
/content/drive/MyDrive/Swin UNetr Data/data2/labelsTr/Segmentation_LT_152_ds256.nii.gz

0th validation image hausdorff distances:, [0.2013119946011562, 0.05828622484241824, 0.23498483689835986, 0.1500863983176419, 26.243691820509177, 0.22339636737367774, 0.24416562785484613, 1.1346240904575178, 0.3690089701308508, 0.10911795079680128, 0.25501305753852593, inf, 1.036136130049014, 1.027717962781444, 2.326863285346185, 0.2486745505723249, 1.2245876186623519]

1th validation image Dice scores:, [[0.9648084  0.95338047 0.9223394  0.9218884  0.49357328 0.9110415
  0.93833977 0.4467766  0.18473895 0.71340805 0.8231271  0.23381294
  0.9139734  0.8299985  0.5477707  0.9012494  0.8637844 ]]
/content/drive/MyDrive/Swin UNetr Data/data2/labelsTr/Segmentation_LT_143_ds256.nii.gz


...


0th validation image Dice scores:, [[0.95393276 0.9488512  0.90616477 0.88125664 0.5720339  0.9339897
  0.8708598  0.78470826 0.10779817 0.6996337  0.8375224  0.4608379
  0.93064433 0.74401355 0.17741935 0.94159037 0.85782814]]
/content/drive/MyDrive/Swin UNetr Data/data2/labelsTr/Segmentation_RT_152_ds256.nii.gz

0th validation image hausdorff distances:, [0.20971136676062835, 0.051514566224877115, 0.035448249524577424, 0.0941931962768254, 0.11109744813456925, 0.30626542417155683, 5.029744677342773, 0.08347583781434562, 1.3101293653370916, 0.24381078769410938, 0.16360369707751352, 0.7856545876471884, 1.0484435750080507, 1.2944640790770015, 1.2469531560056708, 1.3351499429252818, 0.5056447594796141]

1th validation image Dice scores:, [[0.96627426 0.9537258  0.8906092  0.9293333  0.5412668  0.90476304
  0.8838991  0.58704454 0.21996616 0.63420427 0.81752133 0.07709751
  0.8156048  0.7910385  0.15564202 0.91212595 0.79882807]]
/content/drive/MyDrive/Swin UNetr Data/data2/labelsTr/Segme

Loading dataset: 100%|██████████| 3/3 [00:04<00:00,  1.58s/it]



0th validation image Dice scores:, [[0.94268566 0.9585252  0.8645892  0.86806345 0.31128404 0.88083684
  0.91729426 0.21270718 0.35820895 0.5668494  0.7030126  0.32702237
  0.9464664  0.7571354  0.19571045 0.90848786 0.899607  ]]
/content/drive/MyDrive/Swin UNetr Data/data2/labelsTr/Segmentation_LT_145_ds256.nii.gz

0th validation image hausdorff distances:, [0.18749915162096187, 0.04645784122867094, 0.9790915914498919, 0.07995107786896682, 0.2812188863539235, 0.3390140367042415, 0.1467955416007313, 0.7591438475417444, 0.6633742786490595, 0.3902460263454907, 2.8192974562615563, 6.667437056431192, 0.11394210595912815, 0.7309595432868876, 2.3637682324290354, 0.601627269335775, 0.6179328574673958]

1th validation image Dice scores:, [[0.9309192  0.94363564 0.6893798  0.74400187 0.1614679  0.8454332
  0.9159459  0.03529412 0.         0.85899705 0.67386585 0.1038961
  0.73668605 0.7656183  0.3571147  0.80323315 0.8235869 ]]
/content/drive/MyDrive/Swin UNetr Data/data2/labelsTr/Segmentation

Loading dataset: 100%|██████████| 3/3 [00:04<00:00,  1.63s/it]



0th validation image Dice scores:, [[0.96780354 0.94604534 0.88386804 0.88469714 0.2730159  0.80721277
  0.8774653  0.34215885 0.3914209  0.4819765  0.73155886 0.17067833
  0.9272075  0.87767726 0.         0.86693496 0.8597341 ]]
/content/drive/MyDrive/Swin UNetr Data/data2/labelsTr/Segmentation_LT_151_ds256.nii.gz

0th validation image hausdorff distances:, [0.1540958083230833, 0.04307402309755889, 3.4269442020134173, 1.4519020060667964, 3.930098833242966, 0.6529459576858107, 0.7353998667684737, 0.6751390840770543, 0.7754117855002921, 0.5298615964679096, 0.713509132837153, 3.311211800372926, 0.3523910517130697, 0.36225176611100557, 5.406901069403063, 0.1078825479934356, 0.5810405674539714]

1th validation image Dice scores:, [[0.9570324  0.93201154 0.8706794  0.8622079  0.14666666 0.88139904
  0.89667475 0.70156354 0.21956088 0.65129685 0.5868714  0.08853119
  0.84541833 0.7448809  0.11857708 0.8919953  0.8671632 ]]
/content/drive/MyDrive/Swin UNetr Data/data2/labelsTr/Segmentation_R

## Statistics Computation

In [ ]:
print(HD.shape)

(18, 17)


In [ ]:
image_names = np.array(["RT_147", "LT_147", "RT_143", "LT_171",
                        "LT_152", "LT_143", "RT_144", "LT_148",
                        "RT_152", "RT_146", "LT_144", "LT_169",
                        "LT_145", "RT_150", "LT_150",
                        "LT_151", "RT_138", "RT_153"])
class_labels = np.array(['background', 'bone', 'malleus', 'incus', 'stapes',
                         'bony_labyr', 'IAC', 'sup_vestib_n', 'inf_vestib_n',
                         'cochlear_n', 'facial_n','chorda_tymp_n', 'ICA',
                         'sinus_dura', 'vestib_aqueduct', 'mandible', 'EAC'])

df_dice = pd.DataFrame(dice, index = image_names, columns = class_labels)
df_HD = pd.DataFrame(HD, index = image_names, columns = class_labels)

In [ ]:
#Compute winsorized mean
def compute_winsorized_stats(dice_hausdorff):
    winsorized_array = np.copy(dice_hausdorff)

    for i in range(dice_hausdorff.shape[1]):
        #Compute 5th and 95th percentile
        p5 = np.nanpercentile(dice_hausdorff[:,i], 5)
        p95 = np.nanpercentile(dice_hausdorff[:,i], 95)

        # Replace value less than 5th and 95th percentile with its closest values
        winsorized_array[:,i][dice_hausdorff[:,i]<p5] = np.min(dice_hausdorff[:,i][dice_hausdorff[:,i]>p5])
        winsorized_array[:,i][dice_hausdorff[:,i]>p95] = np.max(dice_hausdorff[:,i][dice_hausdorff[:,i]<p95])

    w_mean = np.nanmean(winsorized_array, axis = 0)
    w_std =  np.nanstd(winsorized_array, axis = 0)
    mean =   np.nanmean(dice_hausdorff, axis = 0)
    std =    np.nanstd(dice_hausdorff, axis = 0)

    return w_mean, w_std, mean, std

# print(dice)
# compute_winsorized_stats(dice)
w_mean, w_std, mean, std = compute_winsorized_stats(dice)

new_rows = pd.DataFrame(np.array([w_mean, w_std, mean, std]), index = ["w_mean", "w_std", "mean", "std"], columns = class_labels)
df_dice_wstats = pd.concat([df_dice, new_rows])

w_mean, w_std, mean, std = compute_winsorized_stats(HD)

new_rows = pd.DataFrame(np.array([w_mean, w_std, mean, std]), index = ["w_mean", "w_std", "mean", "std"], columns = class_labels)
df_HD_wstats = pd.concat([df_HD, new_rows])


In [ ]:

df_dice_wstats.to_csv(os.path.join(data_dir, "df_dice_2.csv"))
df_HD_wstats.to_csv(os.path.join(data_dir, "df_HD_2.csv"))

